# Facial Recognition With Tensorflow

CSI4106 Artificial Intelligence

Fall 2019

Group #27

***INTRODUCTION***:  
We are going to use a convolutional nerual network to perform facial recgnition using Tensorflow and Dlib.
We have another approach with different framework Keras, after we implemented all the approaches, we will 
compare these two approaches and baseline.
There are serval steps in this notebook as followed:

1.Preprocessing Data using Facial Detection and Alignment.

2.Creating the Facial Embeddings in Tensorflow as input for training classifier. 

3.Training Classifier. 

4.Evaluating Results.

**Pre-processing Data**:

We'll use the face landmark predictor to fin the location of the eyes and bootom lips of a face in the images. These will be used to center align the image.

In [1]:
# Copyright 2015-2016 Carnegie Mellon University
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import cv2
import dlib
import numpy as np

TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

INV_TEMPLATE = np.float32([
    (-0.04099179660567834, -0.008425234314031194, 2.575498465013183),
    (0.04062510634554352, -0.009678089746831375, -1.2534351452524177),
    (0.0003666902601348179, 0.01810332406086298, -0.32206331976076663)])

TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)


class AlignDlib:

    #: Landmark indices corresponding to the inner eyes and bottom lip.
    INNER_EYES_AND_BOTTOM_LIP = [39, 42, 57]

    #: Landmark indices corresponding to the outer eyes and nose.
    OUTER_EYES_AND_NOSE = [36, 45, 33]

    def __init__(self, facePredictor):
      
        assert facePredictor is not None

        # pylint: disable=no-member
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(facePredictor)

    def getAllFaceBoundingBoxes(self, rgbImg):
       
        assert rgbImg is not None

        try:
            return self.detector(rgbImg, 1)
        except Exception as e:  # pylint: disable=broad-except
            print("Warning: {}".format(e))
            # In rare cases, exceptions are thrown.
            return []

    def getLargestFaceBoundingBox(self, rgbImg, skipMulti=False):
       
        assert rgbImg is not None

        faces = self.getAllFaceBoundingBoxes(rgbImg)
        if (not skipMulti and len(faces) > 0) or len(faces) == 1:
            return max(faces, key=lambda rect: rect.width() * rect.height())
        else:
            return None

    def findLandmarks(self, rgbImg, bb):
        
        assert rgbImg is not None
        assert bb is not None

        points = self.predictor(rgbImg, bb)
        # return list(map(lambda p: (p.x, p.y), points.parts()))
        return [(p.x, p.y) for p in points.parts()]

    # pylint: disable=dangerous-default-value
    def align(self, imgDim, rgbImg, bb=None,
              landmarks=None, landmarkIndices=INNER_EYES_AND_BOTTOM_LIP,
              skipMulti=False, scale=1.0):
        
        assert imgDim is not None
        assert rgbImg is not None
        assert landmarkIndices is not None

        if bb is None:
            bb = self.getLargestFaceBoundingBox(rgbImg, skipMulti)
            if bb is None:
                return

        if landmarks is None:
            landmarks = self.findLandmarks(rgbImg, bb)

        npLandmarks = np.float32(landmarks)
        npLandmarkIndices = np.array(landmarkIndices)

        # pylint: disable=maybe-no-member
        H = cv2.getAffineTransform(npLandmarks[npLandmarkIndices],
                                   imgDim * MINMAX_TEMPLATE[npLandmarkIndices] * scale + imgDim * (1 - scale) / 2)
        thumbnail = cv2.warpAffine(rgbImg, H, (imgDim, imgDim))

        return thumbnail


Now, we created a prprocessor for the dataset and try to find the larest face, center align and output the file.

In [17]:
import argparse
import glob
import logging
import multiprocessing as mp
import os
import time
import cv2

logger = logging.getLogger(__name__)

align_dlib = AlignDlib(os.path.join(os.path.dirname("__file__"), 'shape_predictor_68_face_landmarks.dat'))

def main(input_dir, output_dir, crop_dim):
    start_time = time.time()
    pool = mp.Pool(processes=mp.cpu_count())

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_dir in os.listdir(input_dir):
        image_output_dir = os.path.join(output_dir, os.path.basename(os.path.basename(image_dir)))
        if not os.path.exists(image_output_dir):
            os.makedirs(image_output_dir)

    image_paths = glob.glob(os.path.join(input_dir, '**/*.jpg'))
    for index, image_path in enumerate(image_paths):
        image_output_dir = os.path.join(output_dir, os.path.basename(os.path.dirname(image_path)))
        output_path = os.path.join(image_output_dir, os.path.basename(image_path))
        pool.apply_async(preprocess_image, (image_path, output_path, crop_dim))

    pool.close()
    pool.join()
    logger.info('Completed in {} seconds'.format(time.time() - start_time))


def preprocess_image(input_path, output_path, crop_dim):
   
    image = _process_image(input_path, crop_dim)
    if image is not None:
        logger.debug('Writing processed file: {}'.format(output_path))
        cv2.imwrite(output_path, image)
    else:
        logger.warning("Skipping filename: {}".format(input_path))


def _process_image(filename, crop_dim):
    image = None
    aligned_image = None

    image = _buffer_image(filename)

    if image is not None:
        aligned_image = _align_image(image, crop_dim)
    else:
        raise IOError('Error buffering image: {}'.format(filename))

    return aligned_image


def _buffer_image(filename):
    logger.debug('Reading image: {}'.format(filename))
    image = cv2.imread(filename, )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def _align_image(image, crop_dim):
    bb = align_dlib.getLargestFaceBoundingBox(image)
    aligned = align_dlib.align(crop_dim, image, bb, landmarkIndices=AlignDlib.INNER_EYES_AND_BOTTOM_LIP)
    if aligned is not None:
        aligned = cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB)
    return aligned

input_dir="small_face_dataset_copy"
output_dir="output3"
crop_dim=180

if __name__ == '__main__':
    

    main(input_dir, output_dir, 180)


**Creating Embedings in Tensorflow**:

In [1]:
import logging
import os

import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops


def read_data(image_paths, label_list, image_size, batch_size, max_nrof_epochs, num_threads, shuffle, random_flip,
              random_brightness, random_contrast):

    images = ops.convert_to_tensor(image_paths, dtype=tf.string)
    labels = ops.convert_to_tensor(label_list, dtype=tf.int32)

    # Makes an input queue
    input_queue = tf.compat.v1.train.slice_input_producer((images, labels),
                                                num_epochs=max_nrof_epochs, shuffle=shuffle, )

    images_labels = []
    imgs = []
    lbls = []
    for _ in range(num_threads):
        image, label = read_image_from_disk(filename_to_label_tuple=input_queue)
        image = tf.compat.v1.random_crop(image, size=[image_size, image_size, 3])
        image.set_shape((image_size, image_size, 3))
        image = tf.image.per_image_standardization(image)

        if random_flip:
            image = tf.image.random_flip_left_right(image)

        if random_brightness:
            image = tf.image.random_brightness(image, max_delta=0.3)

        if random_contrast:
            image = tf.image.random_contrast(image, lower=0.2, upper=1.8)

        imgs.append(image)
        lbls.append(label)
        images_labels.append([image, label])

    image_batch, label_batch = tf.compat.v1.train.batch_join(images_labels,
                                                   batch_size=batch_size,
                                                   capacity=4 * num_threads,
                                                   enqueue_many=False,
                                                   allow_smaller_final_batch=True)
    return image_batch, label_batch


def read_image_from_disk(filename_to_label_tuple):
    
    label = filename_to_label_tuple[1]
    file_contents = tf.io.read_file(filename_to_label_tuple[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    return example, label


def get_image_paths_and_labels(dataset):
    image_paths_flat = []
    labels_flat = []
    for i in range(int(len(dataset))):
        image_paths_flat += dataset[i].image_paths
        labels_flat += [i] * len(dataset[i].image_paths)
    return image_paths_flat, labels_flat


def get_dataset(input_directory):
    dataset = []

    classes = os.listdir(input_directory)
    classes.sort()
    nrof_classes = len(classes)
    for i in range(nrof_classes):
        class_name = classes[i]
        facedir = os.path.join(input_directory, class_name)
        if os.path.isdir(facedir):
            images = os.listdir(facedir)
            image_paths = [os.path.join(facedir, img) for img in images]
            dataset.append(ImageClass(class_name, image_paths))

    return dataset


def filter_dataset(dataset, min_images_per_label=10):
    filtered_dataset = []
    for i in range(len(dataset)):
        if len(dataset[i].image_paths) < min_images_per_label:
            continue
        else:
            filtered_dataset.append(dataset[i])
    return filtered_dataset


def split_dataset(dataset, split_ratio=0.7):
    train_set = []
    test_set = []
    min_nrof_images = 2
    for cls in dataset:
        paths = cls.image_paths
        np.random.shuffle(paths)
        split = int(round(len(paths) * split_ratio))
        if split < min_nrof_images:
            continue  
        train_set.append(ImageClass(cls.name, paths[0:split]))
        test_set.append(ImageClass(cls.name, paths[split:-1]))
    return train_set, test_set


class ImageClass():
    def __init__(self, name, image_paths):
        self.name = name
        self.image_paths = image_paths

    def __str__(self):
        return self.name + ', ' + str(len(self.image_paths)) + ' images'

    def __len__(self):
        return len(self.image_paths)


In [2]:
import argparse
import logging
import os
import pickle
import sys
import time

import numpy as np
import tensorflow as tf
from sklearn.svm import SVC
from tensorflow.python.platform import gfile


min_num_images_per_label=10
split_ratio=0.7

def train_or_evaluate(input_directory, model_path, classifier_output_path, batch_size, num_threads, num_epochs,
         min_images_per_labels, split_ratio, for_train):
    
    

    start_time = time.time()
    with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=False)) as sess:
        dataset = get_dataset(input_dir)
        dataset = filter_dataset(dataset, min_images_per_label=min_num_images_per_label)
        train_set, test_set = split_dataset(dataset, split_ratio=split_ratio)
        print("train set:")
        print(train_set)
        print("test set:")
        print(test_set)
        if for_train:
            print("Processing...")
            images, labels, class_names = _load_images_and_labels(train_set, image_size=160, batch_size=batch_size,
                                                                  num_threads=num_threads, num_epochs=num_epochs,
                                                                  random_flip=True, random_brightness=True,
                                                                  random_contrast=True)
        else:
            print("Processing...")
            images, labels, class_names = _load_images_and_labels(test_set, image_size=160, batch_size=batch_size,
                                                                  num_threads=num_threads, num_epochs=1)

        _load_model(model_filepath=model_path)

        init_op = tf.compat.v1.group(tf.compat.v1.global_variables_initializer(), tf.compat.v1.local_variables_initializer())
        sess.run(init_op)

        images_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("input:0")
        embedding_layer = tf.compat.v1.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("phase_train:0")

        coord = tf.train.Coordinator()
        threads = tf.compat.v1.train.start_queue_runners(coord=coord, sess=sess)

        emb_array, label_array = _create_embeddings(embedding_layer, images, labels, images_placeholder,
                                                    phase_train_placeholder, sess)

        coord.request_stop()
        coord.join(threads=threads)
        print('Created {} embeddings'.format(len(emb_array)))

        classifier_filename = classifier_output_path
        print("print classifier_filename: ",classifier_filename)
        if for_train:
            print("ready for training model!!")
            model=_train_and_save_classifier(emb_array, label_array, class_names, classifier_filename)
        else:
            print("Start evaluating:")
            _evaluate_classifier(emb_array, label_array, classifier_filename)

        print('Completed in {} seconds'.format(time.time() - start_time))


def _load_images_and_labels(dataset, image_size, batch_size, num_threads, num_epochs, random_flip=False,
                            random_brightness=False, random_contrast=False):
    class_names = [cls.name for cls in dataset]
    image_paths, labels = get_image_paths_and_labels(dataset)
    images, labels = read_data(image_paths, labels, image_size, batch_size, num_epochs, num_threads,
                                         shuffle=False, random_flip=random_flip, random_brightness=random_brightness,
                                         random_contrast=random_contrast)
    return images, labels, class_names


def _load_model(model_filepath):
    
    model_exp = os.path.expanduser(model_filepath)
    if os.path.isfile(model_exp):
        logging.info('Model filename: %s' % model_exp)
        with gfile.FastGFile(model_exp, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
    else:
        logger.error('Missing model file. Exiting')
        sys.exit(-1)

**Creating embeddings**:

In [3]:
def _create_embeddings(embedding_layer, images, labels, images_placeholder, phase_train_placeholder, sess):
   
    emb_array = None
    label_array = None
    try:
        i = 0
        while True:
            batch_images, batch_labels = sess.run([images, labels])
            emb = sess.run(embedding_layer,
                           feed_dict={images_placeholder: batch_images, phase_train_placeholder: False})

            emb_array = np.concatenate([emb_array, emb]) if emb_array is not None else emb
            label_array = np.concatenate([label_array, batch_labels]) if label_array is not None else batch_labels
            i += 1

    except tf.errors.OutOfRangeError:
        pass
    print(emb_array)
    print(label_array)
    return emb_array, label_array

**Train model**:

In [4]:
def _train_and_save_classifier(emb_array, label_array, class_names, classifier_filename_exp):
    print("Begin to train the Classifier...")
    model = SVC(kernel='linear', probability=True, verbose=False)
    model.fit(emb_array, label_array)
    print("model has trained.")
    with open(classifier_filename_exp, 'wb') as outfile:
        pickle.dump((model, class_names), outfile)
    logging.info('Saved classifier model to file "%s"' % classifier_filename_exp)
    print("Already saved the model.")
    


def _evaluate_classifier(emb_array, label_array, classifier_filename):
    print('Evaluating classifier on {} images'.format(len(emb_array)))
    if not os.path.exists(classifier_filename):
        raise ValueError('Pickled classifier not found, have you trained first?')

    with open(classifier_filename, 'rb') as f:
        model, class_names = pickle.load(f)

        predictions = model.predict_proba(emb_array, )
        best_class_indices = np.argmax(predictions, axis=1)
        best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]

        for i in range(len(best_class_indices)):
            print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))

        accuracy = np.mean(np.equal(best_class_indices, label_array))
        print('Accuracy: %.3f' % accuracy)

**Start to train the model**:

In [9]:
input_dir="output3"
model_path="etc/20180408-102900/20180408-102900.pb"
classifier_output_path="output3/classifier.pkl"
batch_size=128
num_threads=16
num_epochs=25
min_images_per_labels=10
split_ratio=0.7
for_train=True
train_or_evaluate(input_dir, model_path, classifier_output_path, batch_size, num_threads, num_epochs,
         min_images_per_labels, split_ratio, for_train)


train set:
[<__main__.ImageClass object at 0x1a4423af98>, <__main__.ImageClass object at 0x1a4423a438>, <__main__.ImageClass object at 0x1a4423a518>]
test set:
[<__main__.ImageClass object at 0x1a4423a400>, <__main__.ImageClass object at 0x1a4423a470>, <__main__.ImageClass object at 0x1a4423a4a8>]
Processing...
[[-0.02048654 -0.01536106  0.0568049  ...  0.03291871 -0.05079955
   0.06184353]
 [-0.02038794 -0.0148505   0.05511736 ...  0.04280039 -0.04343697
   0.06091926]
 [-0.02066521 -0.00276002  0.06054556 ...  0.04658352 -0.05068344
   0.04872278]
 ...
 [-0.02785842 -0.00965914  0.06017835 ...  0.05048555 -0.04606638
   0.05295586]
 [-0.01624691 -0.014614    0.05499867 ...  0.04735652 -0.04329146
   0.04996042]
 [-0.02606407 -0.0109346   0.05612454 ...  0.05505109 -0.04899501
   0.05481054]]
[0 0 0 ... 2 2 2]
Created 8075 embeddings
print classifier_filename:  output2/classifier2.pkl
ready for training model!!
Begin to train the Classifier...
model has trained.
Already saved the mode

**Evaluating the result**:

In [11]:
input_dir="output3"
model_path="etc/20180408-102900/20180408-102900.pb"
classifier_output_path="output3/classifier.pkl"
batch_size=128
num_threads=16
num_epochs=30
min_images_per_labels=10
split_ratio=0.7
for_train=False
train_or_evaluate(input_dir, model_path, classifier_output_path, batch_size, num_threads, num_epochs,min_images_per_labels, split_ratio,for_train)



train set:
[<__main__.ImageClass object at 0x1a47c753c8>, <__main__.ImageClass object at 0x1a47c752e8>, <__main__.ImageClass object at 0x1a47c751d0>]
test set:
[<__main__.ImageClass object at 0x1a47c75240>, <__main__.ImageClass object at 0x1a47c75208>, <__main__.ImageClass object at 0x1a5c039240>]
Processing...
[[-0.01844431 -0.0017178   0.05657679 ...  0.04719041 -0.0503176
   0.04642105]
 [-0.01134449  0.0001009   0.06367481 ...  0.04308666 -0.05804561
   0.04388401]
 [-0.02383668 -0.01343384  0.05959224 ...  0.03883947 -0.04791413
   0.06200982]
 ...
 [-0.02620896 -0.00888847  0.06026456 ...  0.0542508  -0.04071799
   0.05389923]
 [-0.02397148 -0.00954029  0.05563807 ...  0.0505487  -0.04412433
   0.05754378]
 [-0.03237923 -0.00587172  0.05838428 ...  0.05093258 -0.04220064
   0.05360375]]
[0 0 0 ... 2 2 2]
Created 1167 embeddings
print classifier_filename:  output3/classifier.pkl
Start evaluating:
Evaluating classifier on 1167 images
   0  Aaron_Eckhart: 0.949
   1  Aaron_Eckhart: 